In [18]:
import spacy
import scispacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from scispacy.hyponym_detector import HyponymDetector
import pandas as pd
from spacy import displacy
import en_core_sci_sm
import deplacy
import networkx as nx
import matplotlib.pyplot as plt

In [19]:
with open ("/Users/abhinavshinow/Documents/GitHub/KG-Genaration-from-Biomedical-Text-Using-NER/Data_Collection/heart-data.txt","r") as f:
    text=f.read().replace("\n\n"," ").replace("\n"," ")
text=text[:10000]

In [20]:
nlp_jnlpba=spacy.load('en_ner_jnlpba_md')
nlp_craft=spacy.load('en_ner_craft_md')
nlp_bionlp=spacy.load('en_ner_bionlp13cg_md')
nlp_bc5cdr=spacy.load('en_ner_bc5cdr_md')

In [21]:
print("jnlpba-labels",nlp_jnlpba.pipe_labels['ner'])
print("craft-labels",nlp_craft.pipe_labels['ner'])
print("bionlp-labels",nlp_bionlp.pipe_labels['ner'])
print("Bc5cdr-labels",nlp_bc5cdr.pipe_labels['ner'])

jnlpba-labels ['CELL_LINE', 'CELL_TYPE', 'DNA', 'PROTEIN', 'RNA']
craft-labels ['CHEBI', 'CL', 'GGP', 'GO', 'SO', 'TAXON']
bionlp-labels ['AMINO_ACID', 'ANATOMICAL_SYSTEM', 'CANCER', 'CELL', 'CELLULAR_COMPONENT', 'DEVELOPING_ANATOMICAL_STRUCTURE', 'GENE_OR_GENE_PRODUCT', 'IMMATERIAL_ANATOMICAL_ENTITY', 'MULTI_TISSUE_STRUCTURE', 'ORGAN', 'ORGANISM', 'ORGANISM_SUBDIVISION', 'ORGANISM_SUBSTANCE', 'PATHOLOGICAL_FORMATION', 'SIMPLE_CHEMICAL', 'TISSUE']
Bc5cdr-labels ['CHEMICAL', 'DISEASE']


In [22]:
doc1=nlp_jnlpba(text)
doc2=nlp_craft(text)
doc3=nlp_bionlp(text)
doc4=nlp_bc5cdr(text)

In [23]:
for ents in doc1.ents:
    print(ents)

lymphocyte and monocyte types
white blood cells
pathologically activated eosinophilic white blood cells
psychosocial factors
poverty
insulin


In [24]:
html=displacy.render(doc1,style="ent")

In [25]:
html=displacy.render(doc2,style="ent")

In [26]:
html=displacy.render(doc3,style="ent")

In [27]:
html=displacy.render(doc4,style="ent")

In [28]:
entity=[]
lemma=[]
label=[]
docs=[doc1,doc2,doc3,doc4]

#Enitities in core model
for doc in docs:
    for ents in doc.ents:
        entity.append(str(ents))
        label.append(ents.label_)

In [29]:
df = pd.DataFrame({'Entity': entity,'label':label})

In [30]:
df

,Entity,label
0,lymphocyte and monocyte types,CELL_TYPE
1,white blood cells,CELL_TYPE
2,pathologically activated eosinophilic white bl...,CELL_TYPE
3,psychosocial factors,PROTEIN
4,poverty,PROTEIN
...,...,...
347,diabetes mellitus,DISEASE
348,weight loss,DISEASE
349,glucose,CHEMICAL
350,salt,CHEMICAL


In [31]:
df.to_csv('Entity-Labels.csv', header=False, index=False)